In [1]:
import numpy as np

''' Open Python, then build data from csv file by calling
execfile("smash.py")

Then you can check number of games each character is used in, etc
'''

# OPTIONS

skip_pools = 0 # set to true to use only bracket data

# EXTRACT DATA FROM TABLES
''' First get number of datapoints and number of player names '''
    
file = open( 'smashdata.csv' )
numitems = 0
playernames = {}
charnames = {}
game_outcomes = []
tournaments = {}
for line in file:
    line = line.strip()
    parts = line.split(',')
    if parts[-1] == 'pools':
        if skip_pools == 1:
            continue
    tournaments[ parts[1] ] = 1
    numitems = numitems + 1
    playernames[ parts[2] ] = 1
    playernames[ parts[3] ] = 1
    charnames[ parts[4] ] = 1
    charnames[ parts[5] ] = 1
    game_outcomes.append(parts[6])


print("Smash 64 Results from major 2016 tournaments")
print("\n\tData from following tournaments:")
for key in tournaments:
    print('\t\t' + key)

numchars = len(charnames)
numnames = len(playernames)
if skip_pools == 1:
    print("\tIncludes bracket games only")
if skip_pools != 1:
    print("\tIncludes bracket and pools")
print("\tData taken from all recorded games.")
print( '\nNumber of players in data: ' + str(numnames) )
print( 'Number of games in data: ' + str(numitems) )

Smash 64 Results from major 2016 tournaments

	Data from following tournaments:
		genesis 3
		pound
		goml
	Includes bracket and pools
	Data taken from all recorded games.

Number of players in data: 117
Number of games in data: 415


In [2]:
# Next retrieve character-based results
''' Next construct hastables to map from name to number '''

number2name = {}
name2number = {}
whichname = 0
for item in playernames:
    name2number[ item ] = whichname
    number2name[ whichname ] = item
    whichname = whichname + 1


''' Now construct hashtables for characters '''

number2char = {}
char2number = {}
whichchar = 0
for item in charnames:
    char2number[ item ] = whichchar
    number2char[ whichchar ] = item
    whichchar = whichchar + 1

''' Construct adjacency matrices for game stats ??? '''
# GameMat gives number of games played between players.
# CharGameMat[x,y] gives number of games played between characters x and y
# CharOutcomeMat[x,y] gives number of wins of char y over char x

GameMat = np.zeros((numnames, numnames))
CharGameMat = np.zeros((numchars, numchars))
CharOutcomeMat = np.zeros((numchars, numchars))
pool_or_bracket = []

file.close()
file = open( 'smashdata.csv' )
line_number = 0

char_wins = np.zeros((numchars,1))
for line in file:
    line = line.strip()
    parts = line.split(',')
    pool_or_bracket.append(parts[-1])
    if parts[-1] == 'pools':
        if skip_pools == 1:
            continue
    xj = name2number[ parts[2] ]
    yj = name2number[ parts[3] ]
    GameMat[ xj, yj ] = GameMat[ xj, yj ] + 1
    GameMat[ yj, xj ] = GameMat[ yj, xj ] + 1
    xchar = char2number[ parts[4] ]
    ychar = char2number[ parts[5] ]
    winchar = xchar
    losechar = ychar
    if int(parts[6]) == 2:
        winchar = ychar
        losechar = xchar
    #if int(parts[6]) == 1:
    CharOutcomeMat[ losechar, winchar ] = CharOutcomeMat[ losechar, winchar ] + 1
    if winchar != losechar:
        char_wins[ winchar, 0 ] = char_wins[ winchar, 0 ] + 1
    
    CharGameMat[ xchar, ychar ] = CharGameMat[ xchar, ychar ] + 1
    CharGameMat[ ychar, xchar ] = CharGameMat[ ychar, xchar ] + 1
    line_number = line_number + 1

'''
These data structures contain actual tournament game/match data:
    GameMat
    CharGameMat
    
These contain maps from data structures indices to player/character names:
    char2number
    number2char
    name2number
    number2name
    
'''

char1 = []
char2 = []
char_tots = []
outcome_prct = []
num_matches = []
num_outcomes = []
for i in range( len(char2number) ) :
    for j in range( i, len(char2number) ) :
        char1.append(number2char[i])
        char2.append(number2char[j])
        num_matches.append(CharGameMat[ i,j ])
        num_outcomes.append(CharOutcomeMat[ j,i ])
        if CharGameMat[i,j] != 0:
            outcome_prct.append(CharOutcomeMat[j,i]/CharGameMat[ i,j ])
        if CharGameMat[i,j] == 0:
            outcome_prct.append(-5)

for i in range(len(char2number)):
    char_tots.append( [number2char[i],0] )
    for j in range( len(char2number) ):
        char_tots[i][1] = char_tots[i][1] + CharGameMat[ i,j ]

        
# NOW PRINT
print("Matchup percentages \n")
print("char1\tchar2 \t# games \tchar1 wins \tprct win by char 1")
inds = list(reversed(np.argsort(num_matches)))
#inds = range(len(num_matches))
for i in inds:
#   print( char1[i] + '\t' + char2[i] + '\t' + str(num_matches[ i ] ) + '\t' + str(outcome_prct[i]) )
    if num_matches[i] < 1:
        continue
    print( char1[i] + '\t' + char2[i] + '\t' + str(num_matches[ i ] ) + '\t\t' + str(num_outcomes[i]) + '\t\t' + ("%.2f" % outcome_prct[i]) )

char_tots
charnames = []
charnums = []
for j in range(len(char_tots)):
    charnames.append(char_tots[j][0])
    charnums.append(char_tots[j][1])

inds = list(reversed(np.argsort(charnums)))
sorted_data = []
for j in inds:
    sorted_data.append(char_tots[j])

# This time without dittos
char_tots
charnames = []
charnums = []
charwins = char_wins
for j in range(len(char_tots)):
    charnames.append(char_tots[j][0])
    charnums.append(char_tots[j][1] - CharGameMat[j,j])

inds = list(reversed(np.argsort(charnums)))

print("\nCharacter-based wins in recorded games\n(excludes dittos)")
print("char\t\t# games\t\t% wins")
for i in inds:
    #    print( charnames[i] + '\t' + str(charnums[ i ] ) + '\t' + str(charwins[i]) + ("%.2f" % (charwins[i][0]/charnums[i]) ) )
    print( charnames[i] + '\t\t' + str(charnums[ i ] ) + '\t\t' + ("%.2f" % (charwins[i][0]/charnums[i]) ) )

Matchup percentages 

char1	char2 	# games 	char1 wins 	prct win by char 1
pika	pika	74.0		37.0		0.50
falcon	pika	62.0		31.0		0.50
kirby	pika	42.0		16.0		0.38
pika	yoshi	31.0		18.0		0.58
falcon	falcon	30.0		15.0		0.50
pika	fox	21.0		15.0		0.71
falcon	fox	19.0		9.0		0.47
fox	fox	18.0		9.0		0.50
pika	puff	18.0		8.0		0.44
falcon	yoshi	17.0		7.0		0.41
kirby	yoshi	15.0		8.0		0.53
kirby	falcon	13.0		9.0		0.69
falcon	link	9.0		5.0		0.56
falcon	puff	8.0		6.0		0.75
falcon	ness	8.0		6.0		0.75
kirby	puff	8.0		4.0		0.50
yoshi	fox	7.0		2.0		0.29
kirby	fox	7.0		2.0		0.29
yoshi	puff	7.0		6.0		0.86
puff	fox	6.0		3.0		0.50
mario	yoshi	6.0		6.0		1.00
dk	dk	6.0		3.0		0.50
kirby	kirby	6.0		3.0		0.50
ness	yoshi	4.0		1.0		0.25
kirby	link	4.0		3.0		0.75
pika	link	3.0		2.0		0.67
ness	pika	3.0		1.0		0.33
samus	falcon	3.0		0.0		0.00
dk	pika	3.0		0.0		0.00
kirby	mario	3.0		2.0		0.67
mario	puff	3.0		1.0		0.33
samus	link	2.0		2.0		1.00
samus	fox	2.0		0.0		0.00
mario	ness	2.0		1.0		0.50
yoshi	yoshi	2.0		1.0		0.50
p